In [26]:
#Importing libraries
import pandas as pd
import numpy as np
import tensorflow as tf

In [25]:
#Loading dataset
data = pd.read_csv("/content/mobile_price_classification.csv")

## 1. Data Preparation

In [3]:
data.head()

,battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [4]:
data.shape

(2000, 21)

In [5]:
data.columns

Index(['battery_power', 'bluetooth', 'clock_speed', 'dual_sim', 'front_cam',
       '4G', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'primary_camera',
       'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [6]:
data.dtypes

battery_power       int64
bluetooth           int64
clock_speed       float64
dual_sim            int64
front_cam           int64
4G                  int64
int_memory          int64
m_dep             float64
mobile_wt           int64
n_cores             int64
primary_camera      int64
px_height           int64
px_width            int64
ram                 int64
sc_h                int64
sc_w                int64
talk_time           int64
three_g             int64
touch_screen        int64
wifi                int64
price_range         int64
dtype: object

In [7]:
data.isna().sum()

battery_power     0
bluetooth         0
clock_speed       0
dual_sim          0
front_cam         0
4G                0
int_memory        0
m_dep             0
mobile_wt         0
n_cores           0
primary_camera    0
px_height         0
px_width          0
ram               0
sc_h              0
sc_w              0
talk_time         0
three_g           0
touch_screen      0
wifi              0
price_range       0
dtype: int64

In [8]:
data['price_range'].nunique()

4

In [9]:
data['price_range'].value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

The dataset contains no missing values, indicating no need for further data preprocessing.

## 2. Split the Dataset

In [10]:
# Separate features and labels
X = data.drop('price_range', axis=1)
y = data['price_range']

In [11]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 3.  Build the Neural Network

In [12]:
from keras.models import Sequential
from keras.layers import Dense

### Define the Model
Begin by using Sequential from Keras to initialize the model. Incorporate Dense layers with ReLU activation functions for the hidden layers, and for the output layer, utilize softmax activation, given the multi-class classification nature of the task.

In [24]:
def create_model(optimizer = "SGD"):
    model = Sequential()
    model.add(Dense(12,input_shape=(X_train.shape[1],),activation="relu"))
    model.add(Dense(4, activation='softmax'))

    model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer, metrics=['accuracy'])
    return model

### Wrap the Model with KerasClassifier

In [14]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier

In [15]:
model = KerasClassifier(model = create_model, batch_size = 10, verbose = 0)

### Define hyperparameters

In [16]:
optimizer = ['SGD','Adam','Adamax','Nadam']
epochs = [10,20,30]
learn_rate = [0.001,0.01,0.1,0.2,0.3]

param_grid = dict(model__optimizer = optimizer, optimizer__learning_rate=learn_rate,epochs=epochs)
param_grid

{'model__optimizer': ['SGD', 'Adam', 'Adamax', 'Nadam'],
 'optimizer__learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
 'epochs': [10, 20, 30]}

### Finding the best hyperparameters

In [17]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first l

In [23]:
print("Best accuracy: {:.2f}% using parameters: {}".format(grid_result.best_score_ * 100, grid_result.best_params_))

Best accuracy: 62.88% using parameters: {'epochs': 20, 'model__optimizer': 'Nadam', 'optimizer__learning_rate': 0.01}


In [19]:
best_params = grid_result.best_params_
best_optimizer_name = best_params['model__optimizer']
best_learning_rate = best_params['optimizer__learning_rate']
best_epochs = best_params['epochs']

## 4. Building the model with hyperparameters


In [20]:
from keras.optimizers import Nadam

best_model = Sequential([
    Dense(12, input_shape=(X_train.shape[1],), activation="relu"),
    Dense(4, activation='softmax')  # Suitable for a classification task with 4 classes
])

best_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=Nadam(learning_rate=best_learning_rate),
    metrics=['accuracy']
)

best_model.fit(X_train, y_train, epochs=best_epochs, batch_size=10, verbose=1)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3695 - loss: 189.2521
Epoch 2/20
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4743 - loss: 12.6778
Epoch 3/20
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4922 - loss: 9.1812
Epoch 4/20
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5392 - loss: 7.7694
Epoch 5/20
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5638 - loss: 6.2541
Epoch 6/20
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5528 - loss: 5.3721
Epoch 7/20
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5633 - loss: 5.3822
Epoch 8/20
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6141 - loss: 3.9230
Epoch 9/20
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5953 - loss: 4.6564
Epoch 10/20
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6181 - loss: 4.0732
Epoch 11/20
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6268 - loss: 3.8238
Epoch 12/20
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accur

## 5. Evaluating the Model

In [21]:
y_pred = best_model.predict(X_test)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [22]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Test Loss: 1.484683871269226
Test Accuracy: 0.6549999713897705
